In [32]:
import csv
import json
# from annotation.adjudication.utils import read_turkle_annotation_multiple_scene
# from annotation.analysis.utils.data_util import collect_mentions
csv.field_size_limit(1131072)

1131072

In [42]:
def collect_mentions(instance):
    """
    Collect answer mentions
    """
    answer_spans = instance['answer_spans']
    mentions = []
    for item in answer_spans:
        # Process annotation_result into tuple
        query = (item['querySpan']['sentenceIndex'], item['querySpan']['startToken'], item['querySpan']['endToken'])
        answers = []
        for answer in item['span_list']:
            answers.append((answer['sentenceIndex'], answer['startToken'], answer['endToken']))

        # Add to 6 options
        if item['notMention']:
            mentions.append([query, "notMention"])
        elif item['notPresent']:
            mentions.append([query, "notPresent"])
        else:
            mentions.append([query, answers])
    return mentions


def read_adjudication_results(path):
    """
    Load the annotation_result along with the document
    Output: sentence along with all annotations

    In this version, the input contains annotations from multiple scenes
    Different scenes could be split with sentence_offsets and query_offsets
    """
    output = []
    with open(path, 'r') as f:
        annotation_reader = csv.DictReader(f)
        for instance in annotation_reader:
            inputs = json.loads(instance["Input.json_data"])
            all_sentences = inputs['sentences']
            all_query_spans = inputs['querySpans']
            sentence_offsets = inputs['sentence_offsets']
            query_spans_offsets = inputs['querySpans_offsets']
            all_scene_ids = inputs['scene_ids']
            all_answer_spans = json.loads(instance["Answer.answer_spans"])
            all_option1s = inputs['option1']
            all_option2s = inputs['option2']

            for i in range(len(sentence_offsets) - 1):
                sent_start, sent_end = sentence_offsets[i], sentence_offsets[i + 1]
                query_start, query_end = query_spans_offsets[i], query_spans_offsets[i + 1]
                original_sentences = all_sentences[sent_start: sent_end]
                sentences = []
                for item in original_sentences:
                    temp_sent = []
                    if isinstance(item[0], list):
                        temp_sent.extend(item[0])
                        temp_sent.extend(item[1:])
                        sentences.append(temp_sent)
                    else:
                        sentences.append(item)
                query_spans = all_query_spans[query_start: query_end]
                for item in query_spans:
                    item['sentenceIndex'] -= sentence_offsets[i]

                option1s = all_option1s[query_start: query_end]
                option2s = all_option2s[query_start: query_end]

                answer_spans = all_answer_spans[query_start: query_end]
                for item in answer_spans:
                    item['querySpan']['sentenceIndex'] -= sentence_offsets[i]
                    for x in item['span_list']:
                        if x['startToken'] == -1 and x['endToken'] == -1:
                            continue
                        x['sentenceIndex'] -= sentence_offsets[i]

                temp = {
                    "sentences": sentences,
                    "query_spans": query_spans,
                    "option1": option1s,
                    "option2": option2s,
                    "answer_spans": answer_spans,
                    "WorkerId": instance['WorkerId'],
                    "scene_id": all_scene_ids[i],
                }

                # Collect mentions (For Kappa Cohen)
                answers = collect_mentions(temp)
                temp["answers"] = answers

                # Add to output
                output.append(temp)

    return output


In [43]:
data = []
data.extend(read_adjudication_results('results/result.csv'))

In [ ]:
"span_list"
"querySpan"
"newSure"

In [ ]:
def collect_mentions(instance):
    """
    Collect answer mentions
    """
    answer_spans = instance['answer_spans']
    mentions = []
    for item in answer_spans:
        # Process annotation_result into tuple
        query = (item['querySpan']['sentenceIndex'], item['querySpan']['startToken'], item['querySpan']['endToken'])
        answers = []
        for answer in item['span_list']:
            answers.append((answer['sentenceIndex'], answer['startToken'], answer['endToken']))

        # Add to 6 options
        if item['notMention']:
            mentions.append([query, "notMention"])
        elif item['notPresent']:
            mentions.append([query, "notPresent"])
        else:
            mentions.append([query, answers])
    return mentions

In [113]:
def extract_decision(option):
    """
    Turn to Answer Format
    """
    mentions = option['mentions']
    texts = option['others']
    if mentions:
        if isinstance(mentions[0], dict):
            print(mentions)
        else:
            pass
    else:
        pass
        # print(texts)
    # print()

In [115]:
flag_list = ["notPresent", "notMention", "notSure", "newPresent", "newMention"]
for i in range(len(data)):
    # if i!=20:
    #     continue
    sample = data[i]
    option1 = sample['option1']
    option2 = sample['option2']
    answer_spans = sample['answer_spans']
    # print(len(query_spans), len(option1), len(option2), len(answer_spans))
    for k in range(len(answer_spans)):
        query_span = answer_spans[k]['querySpan']
        query = (query_span['sentenceIndex'], query_span['startToken'], query_span['endToken'])
        if answer_spans[k]['notPresent']: # Annotation A
            decision = option1[k]
            extract_decision(decision)
        elif answer_spans[k]['notMention']: # Annotation B
            decision = option2[k]
        elif answer_spans[k]['notSure']: # Both are Correct
            decision1 = option1[k]
            decision2 = option2[k]
        else:
            drag_selection = answer_spans[k]['span_list'][0]
            if drag_selection['sentenceIndex']!=-1 and drag_selection['startToken']!=-1 and drag_selection['endToken']!=-1: # Select by dragging
                decision = drag_selection
            elif answer_spans[k]['newPresent']: # NotPresent
                decision = "notPresent"
            elif answer_spans[k]['newMention']: # NotMention
                decision = "notMention"
            else:
                pass

(2, 2, 3)
(2, 4, 5)
(3, 25, 26)
(7, 12, 13)
(9, 5, 9)
(10, 12, 13)
(10, 14, 17)
(0, 2, 8)
(1, 4, 6)
(2, 17, 18)
(2, 30, 31)
(2, 34, 35)
[{'sentenceIndex': '14', 'startToken': '30', 'endToken': '31'}]
(2, 42, 43)
[{'sentenceIndex': '14', 'startToken': '34', 'endToken': '35'}]
(2, 45, 46)
(3, 7, 10)
(6, 17, 18)
(6, 19, 20)
(8, 17, 18)
(8, 27, 31)
(8, 32, 33)
(9, 4, 5)
(9, 6, 7)
(10, 6, 8)
(0, 26, 27)
(2, 9, 10)
(3, 7, 9)
(3, 35, 37)
(4, 3, 4)
(5, 2, 3)
(5, 39, 41)
(6, 12, 13)
(7, 25, 26)
(7, 28, 30)
(8, 7, 8)
(8, 15, 16)
(8, 24, 25)
(8, 29, 30)
(8, 32, 33)
(8, 38, 39)
(10, 4, 5)
[{'sentenceIndex': '32', 'startToken': '9', 'endToken': '11'}]
(12, 4, 6)
(14, 16, 17)
(14, 18, 19)
(0, 11, 12)
(0, 13, 14)
(1, 7, 10)
(2, 7, 8)
(2, 9, 11)
(2, 12, 14)
(2, 18, 19)
(3, 10, 11)
(4, 6, 7)
[{'sentenceIndex': '38', 'startToken': '2', 'endToken': '3'}]
(4, 16, 17)
[{'sentenceIndex': '38', 'startToken': '2', 'endToken': '3'}]
(4, 18, 19)
[{'sentenceIndex': '38', 'startToken': '2', 'endToken': '3'}]
(4, 

In [46]:
sample = data[0]
for item in sample:
    print(item)
    print(sample[item])
    print()

sentences
[['Leonard', ':', 'Sheldon', ',', 'it', "'s", 'not', 'a', 'great', 'time', '!'], ['Sheldon', ':', 'I', "'m", 'sorry', ',', 'but', 'this', 'is', 'important', '.'], ['Leonard', ':', 'What', 'is', 'it', '?'], ['Sheldon', ':', 'Back', 'to', 'the', 'Future', 'II', 'was', 'in', 'the', 'Back', 'to', 'the', 'Future', 'III', 'case', '...', 'and', 'Back', 'to', 'the', 'Future', 'III', 'was--', 'get', 'this--', 'in', 'the', 'Back', 'to', 'the', 'Future', 'II', 'case', '.'], ['Leonard', ':', 'So', '?'], ['Sheldon', ':', 'So', ',', 'did', 'you', 'do', 'that', ',', 'or', 'am', 'I', 'in', 'the', 'house', 'with', 'an', 'intruder', '?'], ['Leonard', ':', 'Sheldon', ',', 'I', 'got', 'to', 'go', 'inside', '.', 'It', "'s", 'getting', 'rough', 'out', 'here', '.'], ['Sheldon', ':', 'You', "'re", 'dodging', 'the', 'question', ';', 'I', 'knew', 'it', 'was', 'you', '.'], ['Leonard', ':', 'What', 'was', 'what', '?'], ['Sheldon', ':', 'This', 'is', "n't", 'a', 'very', 'good', 'connection', ',', 'but', 

In [47]:
query_spans = sample['query_spans']
option1 = sample['option1']
option2 = sample['option2']
answer_spans = sample['answer_spans']

7 7 7 7


In [40]:
source_data = []
with open('results/result.csv', 'r') as f:
    reader = csv.DictReader(f)
    for item in reader:
        source_data.append(item)

In [41]:
for i in range(len(source_data)):
    if i!=48:
        continue
    sample = source_data[i]
    input_data = json.loads(sample['Input.json_data'])
    answer_spans = json.loads(sample['Answer.answer_spans'])
    for item in answer_spans:
        print(item)
    print("=="*50)
    for item in input_data:
        print(item)
        print(input_data[item])
        print()

{'querySpan': {'sentenceIndex': 0, 'startToken': 2, 'endToken': 10}, 'notPresent': True, 'notMention': False, 'notSure': False, 'newPresent': False, 'newMention': False, 'newSure': False, 'span_list': [{'sentenceIndex': -1, 'startToken': -1, 'endToken': -1}], 'currentAnswerIndex': 0, 'status': 'ok'}
{'querySpan': {'sentenceIndex': 1, 'startToken': 31, 'endToken': 32}, 'notPresent': True, 'notMention': False, 'notSure': False, 'newPresent': False, 'newMention': False, 'newSure': False, 'span_list': [{'sentenceIndex': -1, 'startToken': -1, 'endToken': -1}], 'currentAnswerIndex': 0, 'status': 'ok'}
{'querySpan': {'sentenceIndex': 1, 'startToken': 31, 'endToken': 34}, 'notPresent': False, 'notMention': True, 'notSure': False, 'newPresent': False, 'newMention': False, 'newSure': False, 'span_list': [{'sentenceIndex': -1, 'startToken': -1, 'endToken': -1}], 'currentAnswerIndex': 0, 'status': 'ok'}
{'querySpan': {'sentenceIndex': 1, 'startToken': 37, 'endToken': 38}, 'notPresent': False, 'not

In [ ]:
"""
HITId
HITTypeId
Title
CreationTime
MaxAssignments
AssignmentDurationInSeconds
AssignmentId
WorkerId
AcceptTime
SubmitTime
WorkTimeInSeconds
Input.json_data
Answer.answer_spans
Turkle.Username
"""